## Notebook to answer question c: Who won each of the individual awards

In [5]:
# Load datasets
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

players_teams = pd.read_csv("project_data/players_teams.csv")
awards = pd.read_csv("project_data/awards_players.csv")
players = pd.read_csv("project_data/players_clean.csv")

### 1. Rookie of the Year Award

In [6]:
# List of rookie players for each year in the dataset.

rookies_by_year = {}
min_year = players_teams['year'].min()
max_year = players_teams['year'].max()
previous_years_players = set()

for year in range(min_year + 1,max_year + 1):
    current_years = players_teams[players_teams['year'] == year]
    current_years_players = set(current_years['playerID'])

    rookies = current_years_players - previous_years_players
    rookies_by_year[year] = rookies

    previous_years_players.update(current_years_players)

for year, rookies in rookies_by_year.items():
    print(f"Year: {year}")
    print(f"Rookies: {rookies}")

Year: 2
Rookies: {'baranel01w', 'johnsja01w', 'hledeko01w', 'miltode01w', 'erbsu01w', 'byearla01w', 'rileyru01w', 'iveyni01w', 'levanni01w', 'edwarto01w', 'scottol01w', 'whitiva01w', 'andrame01w', 'leslili01w', 'farriba01w', 'korstil01w', 'martima01w', 'spornra01w', 'johnspo01w', 'reidtr01w', 'milleke01w', 'anderke01w', 'johnsvi01w', 'santoke01w', 'taylope01w', 'stockta01w', 'saundja01w', 'moweje01w', 'jacksta02w', 'griffyo01w', 'smithta01w', 'mccrini01w', 'dydekma01w', 'ferdima01w', 'burraal01w', 'reedbr01w', 'milleco01w', 'brumfma01w', 'edwarsi01w', 'kubikni01w', 'robincr01w', 'edwarmi01w', 'dossaci01w', 'lazicka01w', 'mcwilta01w', 'payeka01w', 'azzije01w', 'millsta01w', 'marcimi01w', 'ndiayas01w', 'stiresh01w', 'folklkr01w', 'jacksla01w', 'nygaava01w', 'dixonta01w', 'weathte01w', 'johnssh01w', 'hillec01w', 'bullevi01w', 'barksla01w', 'daleygr01w', 'stepama01w', 'burseja01w', 'malcona01w', 'pettibr01w', 'wautean01w', 'vangomi01w', 'dickeke01w', 'thompti01w', 'beviltu01w', 'smithch03w

In [30]:
# Calculate Performance of Rookies in all years
# Performance Formula : ((Points + Rebounds + Assists + Steals + Blocks) - ((Field Goals Att. - Field Goals Made) + (Free Throws Att. - Free Throws Made) + Turnovers))

rookie_performance = {}

for year, rookies in rookies_by_year.items():
    current_years_stats = players_teams[players_teams['year'] == year]
    for rookie in rookies:
        rookie_stats = current_years_stats[current_years_stats['playerID'] == rookie]
        if not rookie_stats.empty:
            points = rookie_stats['points'].values[0]
            rebounds = rookie_stats['rebounds'].values[0]
            assists = rookie_stats['assists'].values[0]
            steals = rookie_stats['steals'].values[0]
            blocks = rookie_stats['blocks'].values[0]
            fg_att = rookie_stats['fgAttempted'].values[0]
            fg_made = rookie_stats['fgMade'].values[0]
            ft_att = rookie_stats['ftAttempted'].values[0]
            ft_made = rookie_stats['ftMade'].values[0]
            turnovers = rookie_stats['turnovers'].values[0]

            performance = (points + rebounds + assists + steals + blocks) - ((fg_att - fg_made) + (ft_att - ft_made) + turnovers)
            rookie_performance[(year, rookie)] = performance
        else:
            rookie_performance[(year, rookie)] = 0

# Identify Rookie of the Year for each year
rookie_of_the_year = {}

for year in range(min_year + 1, max_year + 1):
    rookies = rookies_by_year[year]
    best_rookie = None
    best_performance = float('-inf')
    for rookie in rookies:
        performance = rookie_performance.get((year, rookie), 0)
        if performance > best_performance:
            best_performance = performance
            best_rookie = rookie
    rookie_of_the_year[year] = (best_rookie, best_performance)

# Display Rookie of the Year for each year
for year, (rookie, performance) in rookie_of_the_year.items():
    print(f"Year: {year}, Rookie of the Year: {rookie}, Performance: {performance}")

# Compare with actual awards data
print("\nComparison with Actual Awards Data:")
actual_awards = awards[awards['award'] == 'Rookie of the Year']
actual_awards = actual_awards[['year', 'playerID']]
actual_awards_dict = dict(zip(actual_awards['year'], actual_awards['playerID']))
for year, (predicted_rookie, _) in rookie_of_the_year.items():
    actual_rookie = actual_awards_dict.get(year)
    if actual_rookie == predicted_rookie:
        result = "Correct"
        print(f"Year: {year}, Predicted: {predicted_rookie}, {result}")
    else:
        result = f"Incorrect (Actual: {actual_rookie})"
        comparison = f"Calculated Performance of Actual Rookie of the Year: {rookie_performance[(year, actual_rookie)]}"
        print(f"Year: {year}, Predicted: {predicted_rookie}, {result}, {comparison}")



Year: 2, Rookie of the Year: griffyo01w, Performance: 726
Year: 3, Rookie of the Year: catchta01w, Performance: 754
Year: 4, Rookie of the Year: fordch01w, Performance: 506
Year: 5, Rookie of the Year: tauradi01w, Performance: 512
Year: 6, Rookie of the Year: johnste01w, Performance: 391
Year: 7, Rookie of the Year: augusse01w, Performance: 536
Year: 8, Rookie of the Year: pricear01w, Performance: 350
Year: 9, Rookie of the Year: parkeca01w, Performance: 799
Year: 10, Rookie of the Year: bonnede01w, Performance: 424

Comparison with Actual Awards Data:
Year: 2, Predicted: griffyo01w, Incorrect (Actual: stileja01w), Calculated Performance of Actual Rookie of the Year: 307
Year: 3, Predicted: catchta01w, Correct
Year: 4, Predicted: fordch01w, Correct
Year: 5, Predicted: tauradi01w, Correct
Year: 6, Predicted: johnste01w, Correct
Year: 7, Predicted: augusse01w, Correct
Year: 8, Predicted: pricear01w, Correct
Year: 9, Predicted: parkeca01w, Correct
Year: 10, Predicted: bonnede01w, Incorrec